In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdfsss/pdf2.pdf
/kaggle/input/pdfsss/pdf1.pdf
/kaggle/input/pdfsss/pdf3.pdf


In [2]:
!pip install tabula-py pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 59.9 MB/s eta 0:00:00


In [3]:
import tabula
import pandas as pd
import os

# Function to convert PDF to CSV
def convert_pdf_to_csv(pdf_path, output_dir, use_lattice=False, use_stream=False):
    """
    Converts a PDF file to CSV using Tabula.
    
    Parameters:
    - pdf_path: Path to the PDF file
    - output_dir: Directory to save the output CSV file
    - use_lattice: Boolean to specify if lattice mode should be used
    - use_stream: Boolean to specify if stream mode should be used
    
    Returns:
    - output_csv: Path to the generated CSV file
    """
    # Define output CSV path
    output_csv = os.path.join(output_dir, os.path.basename(pdf_path).replace(".pdf", ".csv"))

    # Debug: Print which PDF is being converted
    print(f"Converting PDF: {pdf_path} -> {output_csv}")

    try:
        # Use Lattice or Stream mode if specified
        if use_lattice:
            tabula.convert_into(pdf_path, output_csv, output_format="csv", pages="all", lattice=True)
        elif use_stream:
            tabula.convert_into(pdf_path, output_csv, output_format="csv", pages="all", stream=True)
        else:
            tabula.convert_into(pdf_path, output_csv, output_format="csv", pages="all")
        
        # Check if the CSV file was created and is not empty
        if os.path.getsize(output_csv) > 0:
            print(f"CSV {output_csv} created successfully.")
        else:
            print(f"Warning: CSV {output_csv} is empty.")

    except Exception as e:
        print(f"Error while converting {pdf_path}: {e}")

    return output_csv

# Function to import CSV files into Excel
def import_csvs_to_excel(csv_files, excel_path):
    """
    Imports CSV files into an Excel workbook, each CSV in a separate sheet.
    
    Parameters:
    - csv_files: List of paths to the CSV files
    - excel_path: Path to the Excel file
    """
    # Create an Excel writer object
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Loop through each CSV file
        for csv_file in csv_files:
            try:
                # Read CSV into a pandas DataFrame
                df = pd.read_csv(csv_file)
                
                # Check if the DataFrame is empty
                if df.empty:
                    print(f"Warning: {csv_file} is empty or cannot be parsed.")
                else:
                    # Generate a sheet name based on the file name (without extension)
                    sheet_name = os.path.basename(csv_file).replace('.csv', '')
                    
                    # Write the DataFrame to a new sheet in the Excel workbook
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"Data from {csv_file} has been imported into Excel sheet '{sheet_name}'.")

            except pd.errors.EmptyDataError:
                print(f"Error: {csv_file} is empty or has no data to parse.")
            except Exception as e:
                print(f"Error while importing {csv_file} to Excel: {e}")

    print(f"All CSV files have been imported into {excel_path}")

# Main function to automate PDF to Excel conversion
def automate_pdf_to_excel(pdf_files, output_directory, excel_file, use_lattice=False, use_stream=False):
    """
    Automates the process of converting PDFs to CSV and importing them into Excel.
    
    Parameters:
    - pdf_files: List of paths to the PDF files
    - output_directory: Directory to save the converted CSVs
    - excel_file: Path to the final Excel file
    - use_lattice: Boolean to specify if lattice mode should be used in Tabula
    - use_stream: Boolean to specify if stream mode should be used in Tabula
    """
    # List to store paths of converted CSV files
    csv_files = []
    
    # Convert each PDF to CSV and store the CSV paths
    for pdf_file in pdf_files:
        csv_file = convert_pdf_to_csv(pdf_file, output_directory, use_lattice, use_stream)
        csv_files.append(csv_file)
    
    # Import all CSV files into the Excel file
    import_csvs_to_excel(csv_files, excel_file)

# Paths to the PDF files (replace with your actual file paths)
pdf_files = [
    "/kaggle/input/pdfsss/pdf1.pdf",
    "/kaggle/input/pdfsss/pdf2.pdf",
    "/kaggle/input/pdfsss/pdf3.pdf"
]

# Directory where the converted CSVs will be saved
output_directory = "/kaggle/working/"

# Path to the final Excel file where the data will be stored
excel_file = "/kaggle/working/imported_data.xlsx"

# Run the automation process (choose lattice or stream mode depending on your PDFs)
automate_pdf_to_excel(pdf_files, output_directory, excel_file, use_lattice=False, use_stream=True)


Converting PDF: /kaggle/input/pdfsss/pdf1.pdf -> /kaggle/working/pdf1.csv
CSV /kaggle/working/pdf1.csv created successfully.
Converting PDF: /kaggle/input/pdfsss/pdf2.pdf -> /kaggle/working/pdf2.csv
CSV /kaggle/working/pdf2.csv created successfully.
Converting PDF: /kaggle/input/pdfsss/pdf3.pdf -> /kaggle/working/pdf3.csv
Data from /kaggle/working/pdf1.csv has been imported into Excel sheet 'pdf1'.
Data from /kaggle/working/pdf2.csv has been imported into Excel sheet 'pdf2'.
Error: /kaggle/working/pdf3.csv is empty or has no data to parse.
All CSV files have been imported into /kaggle/working/imported_data.xlsx
